In [11]:
import pandas as pd
import numpy as np
import tqdm as tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
import xgboost

In [13]:
train=pd.read_csv('/Users/eugeny/Downloads/train-2.csv')

In [14]:
test=pd.read_csv('/Users/eugeny/Downloads/Xtest.csv')

In [15]:
sample=pd.read_csv('/Users/eugeny/Downloads/sample_submission.csv')

In [17]:
train['Password'] = train['Password'].astype(str)
test['Password'] = test['Password'].astype(str)

In [18]:
train['len'] = train['Password'].apply(lambda x: len(x))
train['letter'] = train['Password'].apply(lambda x: sum(c.isalpha() for c in x))
train['digit'] = train['Password'].apply(lambda x: sum(c.isdigit() for c in x))
train['letter_percent']=train['letter']/train['len']
train['digit_percent']=train['digit']/train['len']
train['other'] = train['len'] - train['letter'] - train['digit']
train['up'] = train['Password'].apply(lambda x: sum(c.isupper() for c in x))
train['low'] = train['Password'].apply(lambda x: sum(c.islower() for c in x))

In [19]:
train.head()

,Password,Times,len,letter,digit,letter_percent,digit_percent,other,up,low
0,631XniVx2lS5I,2,13,8,5,0.615385,0.384615,0,4,4
1,LEGIT747,1,8,5,3,0.625000,0.375000,0,5,0
2,742364es,1,8,2,6,0.250000,0.750000,0,0,2
3,3846696477,1,10,0,10,0.000000,1.000000,0,0,0
4,laurahop,2,8,8,0,1.000000,0.000000,0,0,8


In [20]:
test['len'] = test['Password'].apply(lambda x: len(x))
test['letter'] = test['Password'].apply(lambda x: sum(c.isalpha() for c in x))
test['digit'] = test['Password'].apply(lambda x: sum(c.isdigit() for c in x))
test['letter_percent']=test['letter']/test['len']
test['digit_percent']=test['digit']/test['len']
test['other'] = test['len'] - test['letter'] - train['digit']
test['up'] = test['Password'].apply(lambda x: sum(c.isupper() for c in x))
test['low'] = test['Password'].apply(lambda x: sum(c.islower() for c in x))

In [21]:
test.head()

,Id,Password,len,letter,digit,letter_percent,digit_percent,other,up,low
0,0,ThaisCunha,10,10,0,1.000000,0.000000,-5.0,2,8
1,1,697775113,9,0,9,0.000000,1.000000,6.0,0,0
2,2,922a16922a,10,2,8,0.200000,0.800000,2.0,0,2
3,3,andy74,6,4,2,0.666667,0.333333,-8.0,0,4
4,4,joemack,7,7,0,1.000000,0.000000,0.0,0,7


In [22]:
X = train.drop(['Times', 'Password'], axis=1)
y = train['Times']

In [23]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

In [24]:
y_train_log,y_test_log=np.log1p(y_train),np.log1p(y_test)

In [25]:
tree = RandomForestRegressor(random_state=1, n_jobs=-1)

In [26]:
tree_params = {'n_estimators': [i for i in range(10,60,10)]}

In [27]:
tree_params

{'n_estimators': [10, 20, 30, 40, 50]}

In [28]:
grid = GridSearchCV(tree, tree_params, scoring='neg_mean_squared_error', n_jobs=-1, cv=5)

In [29]:
grid.fit(x_train, y_train_log)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=-1,
                                             oob_score=False, random_state=1,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=-1,
             param_grid={'n_estimators': [10, 20, 30, 40, 50]},

In [30]:
pr = grid.best_estimator_.predict(test.drop(['Id', 'Password'], axis=1))

In [31]:
y_test=np.expm1(pr)

In [32]:
sample['Times'] = y_test
sample.to_csv('erer.csv', index=False) 